In [10]:
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Extracting music and features
'''
Dataset
We use GTZAN genre collection dataset for classification. 

The dataset consists of 10 genres i.e

Blues
Classical
Country
Disco
Hiphop
Jazz
Metal
Pop
Reggae
Rock
Each genre contains 100 songs. Total dataset: 1000 songs
'''

'\nDataset\nWe use GTZAN genre collection dataset for classification. \n\nThe dataset consists of 10 genres i.e\n\nBlues\nClassical\nCountry\nDisco\nHiphop\nJazz\nMetal\nPop\nReggae\nRock\nEach genre contains 100 songs. Total dataset: 1000 songs\n'

In [3]:
# Extracting the Spectrogram for every Audio

In [ ]:
# cmap = plt.get_cmap('inferno')

# plt.figure(figsize=(10,10))
# genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
# for g in genres:
#     pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
#     for filename in os.listdir(f'./MIR/genres/{g}'):
#         songname = f'./MIR/genres/{g}/{filename}'
#         y, sr = librosa.load(songname, mono=True, duration=5)
#         plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
#         plt.axis('off');
#         plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
#         plt.clf()

In [5]:
'''
Extracting features from Spectrogram
We will extract

Mel-frequency cepstral coefficients (MFCC)(20 in number)
Spectral Centroid,
Zero Crossing Rate
Chroma Frequencies
Spectral Roll-off.
'''

'\nExtracting features from Spectrogram\nWe will extract\n\nMel-frequency cepstral coefficients (MFCC)(20 in number)\nSpectral Centroid,\nZero Crossing Rate\nChroma Frequencies\nSpectral Roll-off.\n'

In [9]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [7]:
# Writing data to csv file

In [8]:
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'./MIR/genres/{g}'):
        songname = f'./MIR/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rmse(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())


In [9]:
# Analysing the Data in Pandas

In [4]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00060.au,0.430894,0.196222,1946.565652,1979.909934,3955.867746,0.097454,-67.770980,111.704184,-34.646105,...,12.295832,-12.477988,1.681278,-5.142068,4.644002,-6.919217,1.040718,-4.736871,-0.660037,blues
1,blues.00082.au,0.338896,0.251350,2141.461656,2168.015560,4627.997015,0.105151,-29.362093,108.667950,-25.573165,...,5.456504,-7.687713,7.410600,-11.319177,7.229288,-9.466552,1.930059,-6.328476,-1.304812,blues
2,blues.00030.au,0.263016,0.170081,1379.081742,2004.000850,3015.831764,0.039376,-206.987590,117.781468,23.256245,...,-8.015467,-17.616342,-8.138554,-8.646157,-15.538988,-15.331506,-9.664872,-10.103310,-17.835100,blues
3,blues.00007.au,0.307921,0.131785,1451.754147,1577.369917,2955.348796,0.061435,-179.395447,136.459244,-26.656359,...,-6.954827,-3.544535,-8.051242,-8.959537,-8.424337,-10.558885,-10.788159,-4.693749,-8.638613,blues
4,blues.00064.au,0.332480,0.117413,2553.232415,2280.128669,5148.102203,0.146852,-85.150250,88.806722,-16.322611,...,8.478453,-19.590226,6.413210,-13.779667,6.112037,-13.154644,3.933456,-7.615454,3.752626,blues


In [11]:
data.shape

(1000, 28)

In [13]:
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)

In [13]:
# Encoding the Labels

In [11]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [15]:
# Scaling the Feature columns

In [14]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

In [17]:
# Dividing data into training and Testing set

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [16]:
len(y_train)

800

In [17]:
len(y_test)

200

In [18]:
X_train[10]

array([-0.23872759, -0.80221223,  0.57624847,  0.78574554,  0.54864439,
        0.37427164, -0.0934803 , -0.5069832 ,  0.0230438 , -1.34003202,
        1.20316953, -0.91778051,  0.81875847, -0.36062507,  1.09436398,
       -0.71326511,  1.35171358, -0.7367306 ,  0.28746604, -1.26171369,
       -0.24115072, -1.32640744, -0.47675604, -0.56288708,  0.74310684,
       -0.11948634])

In [22]:
# Classification with Keras
# Building our Network

In [21]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

In [22]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [23]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)

Epoch 1/20
7/7 [==============================] - 1s 4ms/step - loss: 2.2138 - accuracy: 0.2450
Epoch 2/20
7/7 [==============================] - 0s 5ms/step - loss: 1.8968 - accuracy: 0.3462
Epoch 3/20
7/7 [==============================] - 0s 17ms/step - loss: 1.6577 - accuracy: 0.3975
Epoch 4/20
7/7 [==============================] - 0s 6ms/step - loss: 1.4523 - accuracy: 0.4913
Epoch 5/20
7/7 [==============================] - 0s 4ms/step - loss: 1.2995 - accuracy: 0.5638
Epoch 6/20
7/7 [==============================] - 0s 4ms/step - loss: 1.1966 - accuracy: 0.5975
Epoch 7/20
7/7 [==============================] - 0s 5ms/step - loss: 1.1094 - accuracy: 0.6400
Epoch 8/20
7/7 [==============================] - 0s 6ms/step - loss: 1.0371 - accuracy: 0.6587
Epoch 9/20
7/7 [==============================] - 0s 4ms/step - loss: 0.9709 - accuracy: 0.6837
Epoch 10/20
7/7 [==============================] - 0s 5ms/step - loss: 0.9144 - accuracy: 0.6975
Epoch 11/20
7/7 [=====================

In [24]:
test_loss, test_acc = model.evaluate(X_test,y_test)

7/7 [==============================] - 0s 2ms/step - loss: 1.0635 - accuracy: 0.6800


In [25]:
print('test_acc: ',test_acc)

test_acc:  0.6800000071525574


In [28]:
# Test accuracy is less than training data accuracy. This hints at Overfitting.

In [29]:
# Validating our approach
# Let's set apart 200 samples in our training data to use as a validation set:

In [26]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

In [31]:
# Now let's train our network for 30 epochs:

In [27]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=30,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Epoch 1/30
2/2 [==============================] - 1s 204ms/step - loss: 2.2938 - accuracy: 0.1383 - val_loss: 2.1145 - val_accuracy: 0.2800
Epoch 2/30
2/2 [==============================] - 0s 39ms/step - loss: 2.1036 - accuracy: 0.3217 - val_loss: 1.9805 - val_accuracy: 0.3150
Epoch 3/30
2/2 [==============================] - 0s 33ms/step - loss: 1.9582 - accuracy: 0.3567 - val_loss: 1.8674 - val_accuracy: 0.3400
Epoch 4/30
2/2 [==============================] - 0s 46ms/step - loss: 1.8291 - accuracy: 0.3817 - val_loss: 1.7575 - val_accuracy: 0.3400
Epoch 5/30
2/2 [==============================] - 0s 35ms/step - loss: 1.6974 - accuracy: 0.4017 - val_loss: 1.6590 - val_accuracy: 0.3800
Epoch 6/30
2/2 [==============================] - 0s 48ms/step - loss: 1.5750 - accuracy: 0.4350 - val_loss: 1.5847 - val_accuracy: 0.3800
Epoch 7/30
2/2 [==============================] - 0s 58ms/step - loss: 1.4750 - accuracy: 0.4833 - val_loss: 1.4995 - val_accuracy: 0.4300
Epoch 8/30
2/2 [==========

In [28]:
results

[1.1237671375274658, 0.6499999761581421]

In [34]:
# Predictions on Test Data

In [29]:
predictions = model.predict(X_test)

7/7 [==============================] - 0s 2ms/step


In [30]:
predictions[0].shape

(10,)

In [31]:
np.sum(predictions[0])

0.99999994

In [32]:
np.argmax(predictions[0])

0

In [34]:
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
print(genres[3])
print(genres[y_test[0]])

disco
blues


In [35]:
model.save('model_weights')

INFO:tensorflow:Assets written to: model_weights/assets


In [36]:
model = keras.models.load_model('model_weights')

In [38]:
print(np.argmax(model.predict(X_test)[0]))

7/7 [==============================] - 0s 5ms/step
0
